In [1]:
from __future__ import print_function
import h5py
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import class_weight
from sklearn.metrics import classification_report
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Input, Concatenate
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from keras.utils import plot_model
from keras.models import Model

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder


from keras.utils import multi_gpu_model
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
import copy
import tensorflow as tf
%matplotlib inline

Using TensorFlow backend.
c:\users\mrathbun2018\.conda\envs\mattwork\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\mrathbun2018\.conda\envs\mattwork\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\mrathbun2018\.conda\envs\mattwork\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\mrathbun2018\.con

In [2]:
hdf5_path = 'Data/dataset.hdf5'
subtract_mean = True

hdf5_file = h5py.File(hdf5_path, "r")

if subtract_mean:
    mm = hdf5_file["train_mean"][...,0]
    mm = mm[np.newaxis, ...]

data_num = hdf5_file["train_flow"].shape[0]
    
num_classes = 2
epochs = 30

flow_rows, flow_cols = 298, 17

x_train = hdf5_file["train_flow"][...,0]
if subtract_mean:
    x_train -= mm

y_train = hdf5_file["train_labels"][:,...]
hdf5_file.close()
hdf5_path = 'Data/dataset-IoT.hdf5'
hdf5_file = h5py.File(hdf5_path, "r")


x_test = hdf5_file["IoT_flow"][...,0]
if subtract_mean:
    x_test -= mm

y_test = hdf5_file["labels"][:,...]

hdf5_file.close()

class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train),
                                                 y_train)
d_class_weights = dict(enumerate(class_weights))

input_shape = (x_train.shape[1], x_train.shape[2])
    
    
#y_train = keras.utils.to_categorical(y_train, num_classes)
#y_test = keras.utils.to_categorical(y_test, num_classes)

label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)


In [3]:
size = 64
nb_filters = 32
num_classes = 2
epochs=30
activations= "tanh"
maxlen=298
dropout = 0.24295335733172563
batch_size = 512
pool_size = 3
lr = 0.0012168203727788032
adam = keras.optimizers.Adam(lr=lr)

optim = adam

layers = 3
kfold = StratifiedKFold(n_splits=20, shuffle=True)
cvscores = []
for train, test in kfold.split(x_train, y_train):
    y_train_k = keras.utils.to_categorical(y_train[train], num_classes)
    y_test_k = keras.utils.to_categorical(y_train[test], num_classes)
    model = Sequential()
    model.add(Conv1D(filters=nb_filters,kernel_size=size,input_shape=(input_shape),padding="valid",activation=activations,strides=1))
    for i in range(layers-1):
        model.add(Conv1D(filters=nb_filters,kernel_size=size,padding="valid",activation=activations,strides=1))
        
    model.add(MaxPooling1D(pool_size= pool_size))
    model.add(Flatten())
    model.add(Dropout(dropout))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(dropout))
    model.add(Dense(num_classes, activation='softmax'))
    model.summary()
    try:
        model = multi_gpu_model(model, gpus = 4)
    except:
        pass
    model.compile(loss='binary_crossentropy', optimizer=optim, metrics=['accuracy'])
    model.fit(x_train[train],y_train_k, batch_size=batch_size, epochs=epochs, verbose=0, class_weight=class_weights, shuffle=True)
    scores = model.evaluate(x_train[test], y_test_k, verbose=0)
    print("%s: %.5f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
    
print("%.5f%% (+/- %.5f%%)" % (np.mean(cvscores), np.std(cvscores)))
    


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 235, 32)           34848     
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 172, 32)           65568     
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 109, 32)           65568     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 36, 32)            0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1152)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1152)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)              

In [4]:
cvscores

[88.14919590950012,
 88.00755143165588,
 86.73276901245117,
 87.9603385925293,
 87.58262395858765,
 87.58262395858765,
 87.01605200767517,
 86.921626329422,
 86.59112453460693,
 88.00755143165588,
 86.68555021286011,
 87.29934096336365,
 87.72426843643188,
 89.04626965522766,
 88.00755143165588,
 87.25212216377258,
 88.29084038734436,
 88.09636235237122,
 88.71043920516968,
 87.48228549957275]